In [3]:
# -!- coding: utf-8 -!-
## load library
%matplotlib inline

import pandas as pd
import numpy as np
# import eng_to_ipa as ipa
import panphon as panphon
import gensim
from gensim.models import word2vec, Word2Vec
from scipy import stats
import matplotlib.pyplot as plt
from wordcloud import WordCloud

FONT_PATH = './util_files/font/simkai.ttf'

import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action = 'ignore')

import csv, os, sys, math
import linecache
import pickle
import random
import datetime

#簡轉繁用 
#https://pypi.python.org/pypi/hanziconv/0.2.1
from hanziconv import HanziConv 

#https://stackoverflow.com/questions/14519177/python-exception-handling-line-number
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [7]:
import multiprocessing

def _apply(pool_map_args):
    df = pool_map_args[0]
    func = pool_map_args[1]
    args = pool_map_args[2]
#     print(args)
    
    return df.apply(func,axis = 1, args = args)

def parallel_apply(df, func, args):
    #指定主機有多少CPU，並把df 平均分配size
    num_processes = 16
    chunk_size = int(df.shape[0]/num_processes)

    #將df 拆解成 num_processes 個 chunk 的df list
    chunks = [df.ix[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]
    pool_map_args = [(df, func, args) for df in chunks]
    pool = multiprocessing.Pool(processes = num_processes)
    result = pool.map(_apply, pool_map_args)
    result = pd.concat([i for i in result])
    pool.terminate()
    pool.join()
    return result

In [19]:
# READ MY DATA INTO CSV FILE
msadata = pd.read_csv('./NameData/EthnicChineseMalaysian.csv', parse_dates = True)

In [174]:
msadata.head()

,LastName,FirstName,Year,Age,Race,Name,IPAName,姓氏,Ethnic,FirstName1,...,FN2,FN1_sonin,FN1_muin,FN2_sonin,FN2_muin,FN1_sonin_index,FN2_sonin_index,FN1_muin_index,FN2_muin_index,YearRange
0,TEOH,Han Sen,1977.0,42.0,Chinese,TEOH HAN SEN,hɑn sɛn,張,Hokkien/Teochew,hɑn,...,10448,H,an,S,en,55,57,80,76,1
1,LIM,Yoke Foong,1977.0,42.0,Chinese,LIM YOKE FOONG,joʊk fuŋ,林,Hokkien,joʊk,...,10449,Y,oke,F,oong,56,60,230,250,1
2,ANG,Soo San,1977.0,42.0,Chinese,ANG SOO SAN,su sæn,洪,Teochew,su,...,10450,S,oo,S,an,57,57,231,80,1
3,CALVIN TAN,Kian Sin,1977.0,42.0,Chinese,CALVIN TAN KIAN SIN,ˈkiən sɪn,陳,Hokkien,ˈkiən,...,10451,K,ian,S,in,58,57,21,75,1
4,CHAN,Khan Thai,1977.0,42.0,Chinese,CHAN KHAN THAI,kɑn taɪ,陳,Cantonese,kɑn,...,10452,Kh,an,Th,ai,59,73,80,64,1


In [8]:
def separateFN(FN):
    if ' ' in FN:
        return FN.split(' ')
    else:
        FN = ' ' + FN
        return FN.split(' ')

In [156]:
# READ IPA-Pinyin INTO FILE
pinyinipa = pd.read_csv('./IPA-Pinyin/Pinyin-IPA.csv', parse_dates = True)
ipapinyin = pd.read_csv('./IPA-Pinyin/IPA-PinyinList.csv', parse_dates = True)
consonants = pd.read_csv('./IPA-Pinyin/ipa-consonants.csv', parse_dates = True)
vowels = pd.read_csv('./IPA-Pinyin/ipa-vowels.csv', parse_dates = True)

In [157]:
pinyin = ipapinyin['Pinyin'].tolist()
ipa = ipapinyin['IPA'].tolist()

ipapylist = map(lambda a, b: a + ',' + b, ipa, pinyin)

In [158]:
ipapylist = list(ipapylist)

In [159]:
print(ipapylist)

['p,b', 'pʰ,p', 'm,m', 'f,f', 't,d', 'tʰ,t', 'n,n', 'l,l', 'k,g', 'kʰ,k', 'x,h', 'tɕ,j', 'tɕʰ,q', 'ɕ,x', 'ʈʂ,zh', 'ʈʂʰ,ch', 'ʂ,sh', 'ɻ~ʐ,r', 'ts,z', 'tsʰ,c', 's,s', 'fəŋ,feng', 'tjou,diu', 'twei,dui', 'twən,dun', 'tʰɤ,te', 'ny,nü', 'ly,lü', 'kɤɚ,ger', 'kʰɤ,ke', 'xɤ,he', 'tɕjɛn,jian', 'tɕjʊŋ,jiong', 'tɕʰin,qin', 'ɕɥɛn,xuan', 'ʈʂɤ,zhe', 'ʈʂɨ,zhi', 'ʈʂʰɤ,che', 'ʈʂʰɨ,chi', 'ʂɤ,she', 'ʂɨ,shi', 'ɻɤ,re', 'ɻɨ,ri', 'tsɤ,ze', 'tswo,zuo', 'tsɨ,zi', 'tsʰɤ,ce', 'tsʰɨ,ci', 'sɤ,se', 'sɨ,si', 'a,a', 'o,o', 'ɔ,o', 'ɛ,e', 'ɤ,e', 'e,e', 'ai,ai', 'ei,ei', 'au,ao', 'ou,ou', 'an,an', 'ən,en', 'aŋ,ang', 'əŋ,eng', 'ʊŋ,ong', 'aɚ,er', 'i,i', 'u,u', 'ɻ̩~ʐ̩,i', 'ɹ̩~z̩,i', 'je,ye', 'jou,you', 'jɛn,yan', 'in,yin', 'iŋ,ying', 'jʊŋ,yong', 'u,wu', 'wo,wo', 'wei,wei', 'wən,wen', 'wəŋ,weng', 'y,yu', 'ɥe,yue', 'ɥɛn,yuan', 'yn,yun']


In [78]:
def ipapinyin(row):
    firstname1 = row['FirstName1']
    if firstname1 != '':
        for ipa,py in ipapylist:
#             print(py,ipa)
            if firstname1 in ipa:
                return py
            else:
                return ipa

In [ ]:
msadata['FirstName1'] = parallel_apply(msadata, ipapylist, ())

In [ ]:
# msadata['IPAName'] = msadata.IPAName.apply(lambda x: separateFN(x))
# msadata['FirstName1'] = msadata.IPAName.apply(lambda x: x[0])
# msadata['FirstName2'] = msadata.IPAName.apply(lambda x: x[1])

# msadata['IPAName'] = msadata['FirstName1']  + ' ' + msadata['FirstName2'].apply(lambda x: x)

# Check the IPA list to convert to Pinyin
msadata['FirstName1'] = msadata.FirstName1.apply(lambda x: ipapinyin(x))
# msadata['FirstName2'] = msadata.FirstName2.apply(lambda x: ipapinyin(x))

# msadata['FN1'] = msadata.FirstName.apply(lambda x: character_to_index(x,1))
# msadata['FN2'] = msadata.FirstName.apply(lambda x: character_to_index(x,2))

In [27]:
msadata[['FirstName','FirstName1','FirstName2']]

,FirstName,FirstName1,FirstName2
0,Han Sen,hɑn,sɛn
1,Yoke Foong,joʊk,fuŋ
2,Soo San,su,sæn
3,Kian Sin,ˈkiən,sɪn
4,Khan Thai,kɑn,taɪ
...,...,...,...
30710,Kelvin,,ˈkɛlvɪn
30711,Jing Xian,ʤɪŋ,zjɑn
30712,Yi Kiat,ji,kiat*
30713,Hui Sze,huɪ,ʃi


In [160]:
msadata['Ethnic'].value_counts()

Hokkien                            9617
Cantonese                          5495
Teochew                            4059
Hakka                              2995
Hokkien/Teochew                    2601
Mandarin                           1869
Hokkien/Teochew/Hainanese/Hakka    1808
Others                              476
Hokkien/Hakka                       457
Hokchew                             289
Foochow                             275
Korean                              219
Hainanese                           201
Cantonese/Hakka                     149
Hokkien/Teochew/Cantonese/Hakka      55
Vietnamese                           37
English                              29
Japanese                             25
Taishan                               8
Indonesia                             1
Name: Ethnic, dtype: int64

In [168]:
name = msadata[['FirstName','Ethnic']]

In [171]:
# Add new column
name['FirstName1'] = ''
name['FirstName2'] = ''

In [172]:
name.to_csv('name_ethnic.csv', index = False)

In [173]:
name

,FirstName,Ethnic,FirstName1,FirstName2
0,Han Sen,Hokkien/Teochew,,
1,Yoke Foong,Hokkien,,
2,Soo San,Teochew,,
3,Kian Sin,Hokkien,,
4,Khan Thai,Cantonese,,
...,...,...,...,...
30710,Kelvin,Mandarin,,
30711,Jing Xian,Hokkien,,
30712,Yi Kiat,Hokkien,,
30713,Hui Sze,Hakka,,


In [84]:
def get_ipa_frequency(data):
    result = []
    ipalist = []
    for value in data.FirstName1:
        ipalist.append(value)
    print(ipalist)
    for i in ipalist:
        wordfreq = ipalist.count(w)
        result.append([w,wordfreq])
    return result

In [85]:
frequency = parallel_apply(msadata,get_ipa_frequency,())

['h', 'ɑ', 'n']
['y', 'i', 'a', 'n', '*']
['j', 'o', 'ʊ', 'k']
['j', 'i', 'u', 'n', '*']
['s', 'u']
['b', 'u', 'n']
['s', 'w', 'e', 'e', '*']
['ˈ', 'k', 'i', 'ə', 'n']
['h', 'ɑ', 'n']
['b', 'u', 'n']
['k', 'ɑ', 'n']
['j', 'u', 'ɑ', 'n']
['m', 'ɪ', 'n']
['k', 'i', 'a', 'ʊ']
['f', 'u', 'k', '*']
['ʧ', 'i']
['ʃ', 'ɛ', 'ŋ']
['w', 'a', 'ɪ']
['h', 'ɪ', 'ŋ']
['f', 'o', 'o', 'k', '*']
['ʧ', 'ə', 'ŋ']
[]
['ɛ', 'ŋ']
['k', 'ɑ']
['ʧ', 'ɛ', 'ŋ']
['y', 'e', 'n', 'g', '*']
['s', 'ɛ', 'ŋ']
['t', 'æ', 'k']
['ʤ', 'u']
['z', 'æ', 'n']
['l', 'i']
['h', 'u', 'o', '*']
['s', 'h', 'e', 'a', 'u', '*']
['p', 'e']
['ʧ', 'ɛ', 'ŋ']
['j', 'ɔ', 'ŋ']
['i']
['k', 'a', 'r', '*']
['m', 'ə', 'n']
['k', 'ɑ', 'k']
['y', 'u', 'e', 'k', '*']
['c', 'h', 'i', 'h', '*']
['y', 'i', 'k', '*']
['ˈ', 'ʤ', 'i', 'ə']
['ʧ', 'i']
['w', 'i']
['ˈ', 'ʤ', 'i', 'ə']
['n', 'y', 'a', '*']
[]
['k', 'ɑ', 'k']
['k', 'ɑ', 'k']
['w', 'a', 'ɪ']
['j', 'ɪ', 'n']
['j', 'i']
['l', 'ɛ', 'ŋ']
['j', 'u']
['ʧ', 'ɪ', 'n']
['s', 'ɑ', 'k']
['ʧ', 'i']
['h', '

['l', 'i']
['ʃ', 'u']
['p', 'e', 'k', '*']
['ʧ', 'i']
['ˈ', 'ʤ', 'i', 'ə']
['s', 'e', 'k', '*']
[]
['b', 'u', 'n']
['j', 'u', 'ɑ', 'n']
['k', 'w', 'ɑ', 'ŋ']
['ʧ', 'ɪ', 'n']
['ə', 'n']
['s', 'u']
['s', 'o', 'o', 'i', '*']
['s', 'o', 'o', 'k', '*']
[]
['ʧ', 'i']
['j', 'æ', 'ŋ']
['h', 'u', 'ɪ']
['p', 'e']
['c', 'h', 'e', 'a', 'h', '*']
['ʃ', 'ə', 'r']
['l', 'i']
['k', 'ɪ', 'm']
['k', 'ɑ']
['w', 'e']
['m', 'ɪ', 'n']
['w', 'a', 'ɪ']
['k', 'ɑ', 'ŋ']
['h', 'ə', 'n']
['ɪ', 'ŋ']
['s', 'u']
['ʤ', 'i']
['k', 'ə', 'r']
['p', 'u', 'i', '*']
['y', 'e', 'a', 'n', '*']
['p', 'ɪ', 'ŋ']
['h', 'o', 'o', 'i', '*']
['p', 'e', 'y', '*']
['l', 'i']
['w', 'ɛ', 'n']
['w', 'e']
['ʧ', 'ɪ', 'n']
['k', 'ɑ']
['w', 'e']
['j', 'u', 'e', 'n', '*']
['t', 'i', 'o', 'n', 'g', '*']
['z', 'e']
['s', 'e', 'o', 'k', '*']
['j', 'ɪ', 'ŋ']
['k', 'ɑ', 'ŋ']
['s', 'u']
['t', 'z', 'e', '*']
['ʧ', 'ə', 'n']
['w', 'e', 'o', 'i', '*']
[]
['b', 'u', 'n']
['h', 'w', 'e', 'e', '*']
['z', 'i']
['ʧ', 'i']
['k', 'æ', 'm']
['w', 'e']
['w', '

['x', 'i', 'n', '*']
['z', 'u', 'o', '*']
[]
['j', 'ə', 'ŋ']
['ʧ', 'i']
['ʃ', 'ɛ', 'n']
['ʧ', 'i']
['k', 'ɑ', 'k']
['w', 'ɛ', 'n']
['ʃ', 'u']
['ʃ', 'i']
['p', 'o', 'ʊ']
['c', 'h', 'o', 'r', '*']
['s', 'z', 'u', '*']
['f', 'e', 'i', '*']
['h', 'ɑ', 'k']
['m', 'e']
['t', 's', 'i', 'n', 'g', '*']
['s', 'i']
['t', 'ɛ', 'ŋ']
['m', 'ɪ', 'ŋ']
['k', 'ɑ', 'k']
['ˈ', 'ʧ', 'i', 'ə']
['j', 'e', 'n', 'g', '*']
['k', 'a', 'r', '*']
['ʧ', 'i']
['f', 'æ', 'ŋ']
['m', 'e']
['ʧ', 'ə', 'r', 'n']
['w', 'e']
['ʃ', 'i', 'ɑ', 'ŋ']
['j', 'i']
['j', 'i']
['h', 'ɔ', 'ŋ']
['p', 'e']
['h', 'o', 'o', 'i', '*']
['w', 'e']
['ʧ', 'ɛ', 'ŋ']
['w', 'e']
['ʧ', 'ɪ', 'n']
['t', 'z', 'y', 'y', '*']
['j', 'u']
['w', 'ɛ', 'n']
['ɪ', 'ŋ']
['x', 'i', 'n', '*']
['p', 'æ', 'ŋ']
['ʃ', 'ə', 'r']
['s', 'h', 'u', 'k', '*']
['h', 'u', 'ɪ']
['ˈ', 'ʤ', 'i', 'ə']
['w', 'e']
['h', 'a', 'ʊ']
['w', 'u', 'n', '*']
['w', 'e']
['j', 'u']
['m', 'ɪ', 'ŋ']
['ˈ', 'ʤ', 'i', 'ə']
['s', 'w', 'e', 'e', '*']
[]
['w', 'ɛ', 'ŋ']
['a', 'i', 'k', '*']
['h',

['k', 'a', 'ɪ']
['k', 'i', 'a', 'ʊ']
[]
['j', 'æ', 'ŋ']
['w', 'e']
['h', 'i']
['s', 'u']
['s', 'u']
['ʧ', 'u']
['x', 'i', 'n', '*']
['s', 'u']
['g', 'e', 'o', 'k', '*']
['ʃ', 'ɛ', 'ŋ']
['g', 'j', 'e', 'n', '*']
['w', 'a', 'ɪ']
['j', 'i']
['ʧ', 'a', 'ʊ']
['l', 'e']
['ʃ', 'o', 'ʊ']
['s', 'ɛ', 'ŋ']
['k', 'ɑ', 'k']
['ʧ', 'ɛ', 'n']
['k', 'a', 'r', '*']
['y', 'i', 'k', '*']
['n', 'g', 'e', 'e', '*']
['w', 'ɑ', 'n']
['b', 'a', 'ʊ']
['k', 'ɑ']
['z', 'i']
['k', 'e', 'h', '*']
['s', 'u', 'n']
['ʃ', 'i']
['k', 'ɪ', 't']
['f', 'a', 'h', '*']
['w', 'e']
['ʃ', 'i']
['ʤ', 'ɛ', 'n']
['h', 'u', 'ɪ']
['ʧ', 'ɛ', 'n']
['j', 'i']
['ʃ', 'i']
['z', 'ɛ', 'ŋ']
['s', 'u', 'n']
['h', 'ɛ', 'ŋ']
['h', 'u', 'ɪ']
['p', 'e']
['ʃ', 'o', 'ʊ']
['p', 'e']
['t', 'j', 'ɑ', 'n']
['z', 'i', 'e', '*']
[]
['i']
['p', 'h', 'o', 'o', 'i', '*']
['k', 'ɑ', 'ŋ']
['w', 'e']
['s', 'w', 'i']
['k', 'o', 'ʊ']
['j', 'ɪ', 'n']
['k', 'a', 'ɪ']
['m', 'ɛ', 'ŋ']
['w', 'e']
['k', 'ɑ']
['k', 'u', 'e', 't', '*']
['ʧ', 'ɪ', 'n']
['b', 'u', 'n']
[

['h', 'u', 'ɪ']
['k', 'i', 'n']
['k', 'ɑ', 'k']
[]
['c', 'h', 'u', 'e', 'n', '*']
['h', 's', 'u', 'e', 'h', '*']
['k', 'w', 'ɑ', 'n']
['ʤ', 'ə', 'n']
['m', 'ɪ', 'ŋ']
['m', 'e']
['m', 'e']
['h', 'u', 'ɪ']
['j', 'ɑ']
['ʃ', 'i']
['w', 'e']
['ʧ', 'ɛ', 'n']
[]
['w', 'a', 'ɪ']
['i']
['ʧ', 'ɛ', 'n']
['l', 'i']
['j', 'i']
['h', 'ɔ', 'ŋ']
['k', 'i', 'a', 'n', 'g', '*']
['ʤ', 'ɪ', 'ŋ']
['t', 'o', 'o', 'n', 'g', '*']
['j', 'ɔ', 'ŋ']
['w', 'e']
['k', 'ɪ', 'm']
['k', 'h', 'e', 'n', 'g', '*']
['w', 'ɛ', 'n']
['p', 'e']
['j', 'ɛ', 'n']
['w', 'e']
['h', 'u', 'ɪ']
['t', 'ɪ', 'ŋ']
['l', 'i']
['ˈ', 'ʤ', 'i', 'ə']
['s', 'u']
['s', 'u', 'n']
[]
['ʧ', 'ɪ', 'n']
['g', 'i', 'a', '*']
['h', 'ɑ', 'n']
['t', 'z', 'e', '*']
['ʃ', 'ɪ', 'n']
['z', 'h', 'e', 'n', '*']
['j', 'i']
['l', 'i']
['p', 'ɛ', 'ŋ']
['w', 'a', 'ɪ']
['s', 'i', 'e', 'z', '*']
['m', 'ɛ', 'ŋ']
['w', 'ɛ', 'ŋ']
['w', 'e']
['t', 'z', 'e', '*']
['ʤ', 'ə', 'n']
['ʧ', 'æ', 'ŋ']
['l', 'ɪ', 'ŋ']
['y', 'u', 't', '*']
[]
['ʧ', 'i']
['j', 'ɔ', 'ŋ']
['ʧ', 'i'

...

Process ForkPoolWorker-266:
Process ForkPoolWorker-261:
Process ForkPoolWorker-268:
Process ForkPoolWorker-270:
Process ForkPoolWorker-258:
Process ForkPoolWorker-267:
Process ForkPoolWorker-257:
Process ForkPoolWorker-264:
Process ForkPoolWorker-260:
Process ForkPoolWorker-272:
Process ForkPoolWorker-269:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr

  File "<ipython-input-7-3e6e46c0f1b2>", line 9, in _apply
    return df.apply(func,axis = 1, args = args)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-7-3e6e46c0f1b2>", line 9, in _apply
    return df.apply(func,axis = 1, args = args)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/pandas/core/frame.py", line 6928, in apply
    return op.get_result()
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/pandas/core/frame.py", line 6928, in apply
    return op.get_result()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/pandas/core/frame.py", line 6928, in apply
    return op.get_result()
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/pandas/core/frame.py

  File "pandas/_libs/reduction.pyx", line 153, in pandas._libs.reduction.Reducer.get_result
  File "pandas/_libs/reduction.pyx", line 651, in pandas._libs.reduction.reduce
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/pandas/core/apply.py", line 285, in apply_standard
    values, self.f, axis=self.axis, dummy=dummy, labels=labels
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/pandas/core/apply.py", line 285, in apply_standard
    values, self.f, axis=self.axis, dummy=dummy, labels=labels
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/pandas/core/apply.py", line 186, in get_result
    return self.apply_standard()
  File "pandas/_libs/reduction.pyx", line 651, in pandas._libs.reduction.reduce
  File "pandas/_libs/reduction.pyx", line 651, in pandas._libs.reduction.reduce
  File "pandas/_libs/reduction.pyx", line 651, in pandas._libs.reduction.reduce
  File "pandas/_libs/reduction.pyx", line 651, in pandas._libs.reduction.reduce
  File "/home/yvonne

  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 356, in flush
    self._flush()
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/jupyter_client/session.py", line 748, in send
    stream.send_multipart(to_send, copy=copy)
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 356, in flush
    self._flush()
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 382, in _flush
    parent=self.parent_header, ident=self.topic)
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 356, in flush
    self._flush()
KeyboardInterrupt
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 356, in flush
    self._flush()
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 382, in _flush
    parent=self.parent_header, ident=self.topic)
  File "/home/yvonnechow/.local/lib/pyth

  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 207, in schedule
    f()
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/jupyter_client/session.py", line 750, in send
    if self.debug:
KeyboardInterrupt
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 214, in send_multipart
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "zmq/backend/cython/context.pyx", line 91, in zmq.backend.cython.context.Context.term
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 214, in <lambda>
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 230, in _really_send
    ctx.term()
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel/iostream.py", line 207, in schedule
    f()
  File "/home/yvonnechow/.local/lib/python3.6/site-packages/ipykern